# 🖊️ SignVerifAI - Model Upgrade v4.0

**New Features:**
- 🎯 **ArcFace Loss**: Angular margin for better separation
- 🏗️ **Multiple Backbones**: MobileNetV3, ResNet50, EfficientNet-B0
- 🔥 **Batch-Hard Triplet Mining**: True hard example mining
- 🧊 **Freeze/Unfreeze**: 3 epoch backbone freeze
- ⚡ **Mixed Precision (AMP)**

**Target:**
- AUC >= 0.95
- EER <= 0.08
- FAR@FRR=1% <= 0.10

---

## 1️⃣ GPU Check & Setup

In [ ]:
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ Clone & Install

In [ ]:
!rm -rf SignVerifAI
!git clone https://github.com/gorkemelih/SignVerifAI.git
%cd SignVerifAI
!pip install -e . -q
print("✅ Kurulum tamamlandı!")

## 3️⃣ Mount Google Drive & Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
DRIVE_DATA_PATH = "/content/drive/MyDrive/SignVerifAI/data_processed.zip"

if os.path.exists(DRIVE_DATA_PATH):
    !unzip -q -o {DRIVE_DATA_PATH} -d .
    print("✅ Veriler yüklendi!")
else:
    print(f"❌ Dosya bulunamadı: {DRIVE_DATA_PATH}")

## 4️⃣ Prepare Data

In [ ]:
!signverify split
!signverify pairs

## 5️⃣ Train Model 🚀

**Key Options:**
- `--loss arcface`: ArcFace loss with angular margin
- `--backbone mobilenet_v3_large`: or resnet50, efficientnet_b0
- `--batch-hard`: True batch-hard triplet mining

In [ ]:
# Training with ArcFace loss (recommended)
!signverify train \
    --device cuda \
    --epochs 40 \
    --batch-size 128 \
    --backbone mobilenet_v3_large \
    --freeze-epochs 3 \
    --backbone-lr 1e-5 \
    --head-lr 3e-4 \
    --loss arcface \
    --batch-hard \
    --scheduler onecycle \
    --log-every 5

### Alternative: ResNet50 (higher capacity)

In [ ]:
# Uncomment to try ResNet50 backbone
# !signverify train \
#     --device cuda \
#     --epochs 40 \
#     --backbone resnet50 \
#     --loss arcface

## 6️⃣ Evaluate

In [ ]:
!signverify eval --device cuda --threshold-mode all

In [ ]:
from IPython.display import Image, display, Markdown

print("📊 ROC Curve:")
display(Image('outputs/reports/roc_curve.png'))

print("\n📊 Score Distribution:")
display(Image('outputs/reports/score_distribution.png'))

In [ ]:
with open('outputs/reports/eval_report.md', 'r') as f:
    display(Markdown(f.read()))

## 7️⃣ Training History

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

history = pd.read_csv('outputs/reports/train_history.csv')

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0,0].plot(history['epoch'], history['train_loss'], label='Train')
axes[0,0].plot(history['epoch'], history['val_loss'], label='Val')
axes[0,0].set_title('Loss')
axes[0,0].legend()

axes[0,1].plot(history['epoch'], history['auc'])
axes[0,1].axhline(y=0.95, color='r', linestyle='--', label='Target')
axes[0,1].set_title('AUC')
axes[0,1].legend()

axes[1,0].plot(history['epoch'], history['eer'])
axes[1,0].axhline(y=0.08, color='r', linestyle='--', label='Target')
axes[1,0].set_title('EER (lower is better)')
axes[1,0].legend()

axes[1,1].plot(history['epoch'], history['lr'])
axes[1,1].set_title('Learning Rate')

plt.tight_layout()
plt.show()

## 8️⃣ Save to Drive

In [ ]:
import shutil
import glob

runs = sorted(glob.glob('outputs/models/run_*'))
if runs:
    latest_run = runs[-1]
    dest = "/content/drive/MyDrive/SignVerifAI/trained_models/"
    os.makedirs(dest, exist_ok=True)
    
    for f in glob.glob(f"{latest_run}/*"):
        shutil.copy(f, dest)
    
    reports_dest = "/content/drive/MyDrive/SignVerifAI/reports/"
    os.makedirs(reports_dest, exist_ok=True)
    for f in glob.glob('outputs/reports/*'):
        shutil.copy(f, reports_dest)
    
    print(f"✅ Saved to Drive!")

---
## 📋 v4.0 Config Summary

| Setting | Value |
|---------|-------|
| Epochs | 40 |
| Batch Size | 128 |
| Loss | **ArcFace** (s=30, m=0.5) |
| Backbone | MobileNetV3-Large |
| Freeze | 3 epochs |
| Backbone LR | 1e-5 |
| Head LR | 3e-4 |
| Scheduler | OneCycleLR |
| Batch-Hard | ✅ |